# Bank Note Classification Using Multi Kernel SVM Learning

Importing all the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from prettytable import PrettyTable
import math
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Reading the file
df = pd.read_csv('./data_banknote_authentication.txt', header=None)

# renaming the features for better understanding
df = df.set_axis(['Wavelet Variance', 'Wavelet Skewness', 'Wavelet Curtosis', 'Image Entropy', 'Label'], axis=1,inplace=False)

# glimpse of the dataframe
df.head()

,Wavelet Variance,Wavelet Skewness,Wavelet Curtosis,Image Entropy,Label
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [ ]:
# Seperating features and their lables
X = df.iloc[:,:4]
y = df.iloc[:,4]

Train Test splitting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.40, random_state=8)

In [ ]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
X_test = X_test.to_numpy()
np.random.seed(45)

Linear SVM

In [ ]:
def linearSVM(X_train, y_train, X_test, y_test, C):
    classifier = SVC(C=C, kernel = 'linear')
    classifier.fit(X_train, y_train)

    # adding results to myTable
    myTable.add_row(['Linear', f'{C}', 'Not Applicable', \
        f'{classifier.score(X_test, y_test):.4f}'])

Polynomial SVM

In [ ]:
def polynomialSVM(X_train, y_train, X_test, y_test, C, degree):
    classifier = SVC(C=C, kernel = 'poly', degree=degree)
    classifier.fit(X_train, y_train)

    # adding results to myTable
    myTable.add_row(['Polynomial', f'{C}', f'{degree}',\
        f'{classifier.score(X_test, y_test):.4f}'])

RBF SVM

In [ ]:
def rbfSVM(X_train, y_train, X_test, y_test, C):
    classifier = SVC(C=C, kernel = 'rbf')
    classifier.fit(X_train, y_train)

    # adding results to myTable
    myTable.add_row(['rbf', f'{C}', 'Not Applicable', \
        f'{classifier.score(X_test, y_test):.4f}'])

Sigmoid SVM

In [ ]:
def sigmoidSVM(X_train, y_train, X_test, y_test, C):
    classifier = SVC(C=C, kernel = 'sigmoid')
    classifier.fit(X_train, y_train)

    # adding results to myTable
    myTable.add_row(['sigmoid', f'{C}', 'Not Applicable', \
        f'{classifier.score(X_test, y_test):.4f}'])

Precomputed SVM

In [ ]:
def precomputedSVM(X_train, y_train, X_test, y_test, C):
    new_train = np.dot(X_train, X_train.T)
    new_test = np.dot(X_test, X_train.T)
    classifier = SVC(C=C, kernel = 'precomputed')
    classifier.fit(new_train, y_train)
    
    # adding results to myTable
    myTable.add_row(['Precomputed', f'{C}', 'Not Applicable', \
        f'{classifier.score(new_test, y_test):.4f}'])

Multi Kernel SVM (linear combination of Linear, rbf and polynomial kernel)

In [ ]:
# user defined function for polynomial kernel
def polynomial(u, v):
    return (np.dot(u, v) )**2

# user defined function for rbf kernel
def rbf(u, v):
    w = u - v
    return math.exp( -3 * np.dot(w, w))

# user defined function for Mutli-kernel
def multikernel(X_train,y):
    
    #linear Kernel
    k1=np.dot(X_train, y.T)

    #rbf kernel
    k2 = np.zeros((X_train.shape[0], y.shape[0]))
    for n, x_n in enumerate(X_train):
        for m, x_m in enumerate(y):
            k2[n,m] = rbf(x_n, x_m)

    # polynomial kernel
    k3=np.zeros((X_train.shape[0], y.shape[0]))
    for n, x_n in enumerate(X_train):
        for m, x_m in enumerate(y):
            k3[n,m] = polynomial(x_n, x_m)

    # taking linear combination of all the kernels
    a = np.random.random()
    b = np.random.random()
    c = (1-a-b)
    return a*k1 + b*k2+ c*k3

In [ ]:
def mklSVM(X_train, y_train, X_test, y_test, C):
    classifier = SVC(C=C,kernel=multikernel)
    classifier.fit(X_train, y_train)

    # adding results to myTable
    myTable.add_row(['MultiKernel', f'{C}', 'Not Applicable', \
        f'{classifier.score(X_test, y_test):.4f}'])

Calling every kernel learning algorithm

In [ ]:
def multiKernelSVM(X_train, y_train, X_test, y_test, C, degree):
    linearSVM(X_train, y_train, X_test, y_test,C)
    polynomialSVM(X_train, y_train, X_test, y_test, C, degree)
    rbfSVM(X_train, y_train, X_test, y_test, C)
    sigmoidSVM(X_train, y_train, X_test, y_test, C)
    precomputedSVM(X_train, y_train, X_test, y_test, C)
    mklSVM(X_train, y_train, X_test, y_test, C)

### Test Case 01

In [ ]:
# Making a Table to store the results
myTable = PrettyTable(['Kernel', 'C', 'Degree', 'Accuracy'])
multiKernelSVM(X_train, y_train, X_test, y_test, C=0.6, degree=2)
print('Results')
print(myTable)

Results
+-------------+-----+----------------+----------+
|    Kernel   |  C  |     Degree     | Accuracy |
+-------------+-----+----------------+----------+
|    Linear   | 0.6 | Not Applicable |  0.9909  |
|  Polynomial | 0.6 |       2        |  0.9472  |
|     rbf     | 0.6 | Not Applicable |  0.9945  |
|   sigmoid   | 0.6 | Not Applicable |  0.6958  |
| Precomputed | 0.6 | Not Applicable |  0.9909  |
| MultiKernel | 0.6 | Not Applicable |  0.9800  |
+-------------+-----+----------------+----------+


### Test Case 02

In [ ]:
# Making a Table to store the results
myTable = PrettyTable(['Kernel', 'C', 'Degree', 'Accuracy'])
multiKernelSVM(X_train, y_train, X_test, y_test, C=0.4,degree=4)
print('Results')
print(myTable)

Results
+-------------+-----+----------------+----------+
|    Kernel   |  C  |     Degree     | Accuracy |
+-------------+-----+----------------+----------+
|    Linear   | 0.4 | Not Applicable |  0.9909  |
|  Polynomial | 0.4 |       4        |  0.8361  |
|     rbf     | 0.4 | Not Applicable |  0.9927  |
|   sigmoid   | 0.4 | Not Applicable |  0.7049  |
| Precomputed | 0.4 | Not Applicable |  0.9909  |
| MultiKernel | 0.4 | Not Applicable |  0.5301  |
+-------------+-----+----------------+----------+


### Test Case 03

In [ ]:
# Making a Table to store the results
myTable = PrettyTable(['Kernel', 'C', 'Degree', 'Accuracy'])
multiKernelSVM(X_train, y_train, X_test, y_test, C=0.9, degree=1)
print('Results')
print(myTable)

Results
+-------------+-----+----------------+----------+
|    Kernel   |  C  |     Degree     | Accuracy |
+-------------+-----+----------------+----------+
|    Linear   | 0.9 | Not Applicable |  0.9854  |
|  Polynomial | 0.9 |       1        |  0.9800  |
|     rbf     | 0.9 | Not Applicable |  0.9945  |
|   sigmoid   | 0.9 | Not Applicable |  0.6885  |
| Precomputed | 0.9 | Not Applicable |  0.9854  |
| MultiKernel | 0.9 | Not Applicable |  0.4444  |
+-------------+-----+----------------+----------+


### Test Case 04

In [ ]:
# Making a Table to store the results
myTable = PrettyTable(['Kernel', 'C', 'Degree', 'Accuracy'])
multiKernelSVM(X_train, y_train, X_test, y_test, C=0.1, degree=2)
print('Results')
print(myTable)

Results
+-------------+-----+----------------+----------+
|    Kernel   |  C  |     Degree     | Accuracy |
+-------------+-----+----------------+----------+
|    Linear   | 0.1 | Not Applicable |  0.9854  |
|  Polynomial | 0.1 |       2        |  0.8944  |
|     rbf     | 0.1 | Not Applicable |  0.9872  |
|   sigmoid   | 0.1 | Not Applicable |  0.7395  |
| Precomputed | 0.1 | Not Applicable |  0.9854  |
| MultiKernel | 0.1 | Not Applicable |  0.9508  |
+-------------+-----+----------------+----------+


### Test Case 05

In [ ]:
# Making a Table to store the results
myTable = PrettyTable(['Kernel', 'C', 'Degree', 'Accuracy'])
multiKernelSVM(X_train, y_train, X_test, y_test, C=0.3, degree=10)
print('Results')
print(myTable)

Results
+-------------+-----+----------------+----------+
|    Kernel   |  C  |     Degree     | Accuracy |
+-------------+-----+----------------+----------+
|    Linear   | 0.3 | Not Applicable |  0.9909  |
|  Polynomial | 0.3 |       10       |  0.7195  |
|     rbf     | 0.3 | Not Applicable |  0.9927  |
|   sigmoid   | 0.3 | Not Applicable |  0.7049  |
| Precomputed | 0.3 | Not Applicable |  0.9909  |
| MultiKernel | 0.3 | Not Applicable |  0.6776  |
+-------------+-----+----------------+----------+
